In [86]:
## Get the number of genes above LOD as a function of nuclei number:
%pylab inline
import pandas as pd
import sys, ast, os
rcParams['axes.spines.right'] = False
rcParams['axes.spines.top'] = False
import pickle as pickle
import numpy as np
import time
import itertools
data_type = 'float32'
os.environ["THEANO_FLAGS"] = 'floatX=' + data_type #+ ',force_device=True'
%matplotlib inline
import anndata
import scanpy as sc
import theano.tensor as tt
import pymc3 as pm
#gfd
import theano
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import warnings
import os
import NaiveDE
import SpatialDE
import statsmodels.stats.multitest as multi
import scipy.stats as ss

os.chdir('/home/jovyan/KR_NAS/')
rSlides = ('00MU', '00MV', '00MV-2')
AOI_type = 'Geometric'
Radial_position = 2

# Let's load the data and metadata from our Nanostring experiment:
counts = pd.read_table('Sanger_288ROIs_TargetCountMatrix.txt')
genes = counts['TargetName']
counts = counts.drop('TargetName',1)
counts = counts.rename(index=genes)
counts = counts.astype('int')
counts = counts[np.array([sum(counts.iloc[i,:]) for i in range(len(counts.iloc[:,0]))]) > 3 * 288]
metadata = pd.read_csv('NanoString sequencing all annotations 2020.02.10_original.csv')
metadata = metadata.iloc[0:286,]
metadata = metadata.rename(index=metadata['Sanger_sampleID'])
metadata = metadata.reindex(np.array(counts.columns))
properties = pd.read_table('Sanger_288ROIs_SegmentProperties.txt')
properties = properties.rename(index=properties['DSP_Sample_ID'])
properties = properties.reindex(np.array(metadata['Sample_ID']))
columnNames = ('x', 'y', 'total_counts', 'Q3_counts')
sample_info = pd.DataFrame(index=metadata['Sample_ID'], columns=columnNames)
sample_info['x'] = np.array(metadata['VCDepth'])
sample_info['y'] = np.array(metadata['Radial_position'])
sample_info['total_counts'] = [sum(counts.iloc[:,i]) for i in range(len(counts.iloc[1,:]))] 
sample_info['Q3_counts'] = [sum(np.sort(counts.iloc[:,i])[int(np.round(0.5*len(counts.iloc[:,i]))):int(np.round(0.75*len(counts.iloc[:,i])))]) for i in range(len(counts.iloc[1,:]))] 

Populating the interactive namespace from numpy and matplotlib


In [88]:
# Let's load the Polioudakis 2019 data as a reference, as well as the cell type specific marker genes:

subset_rois = [metadata.index[i] in np.array(('HCA_F_BRA8739360', 'HCA_F_BRA8739352', 'HCA_F_BRA8739368', 'HCA_F_BRA8739264', 'HCA_F_BRA8739256', 'HCA_F_BRA8739272')) for i in range(len(metadata.index))]
relevantIDs = metadata['Sample_ID'][subset_rois]
relevantSangerIDs = metadata['Sanger_sampleID'][subset_rois]
counts_subset = counts.loc[:,relevantSangerIDs]
properties_subset = properties.loc[np.array(relevantIDs),:]
newLOD_2_subset = properties_subset['NegGeoMean_01']*(properties_subset['NegGeoSD_01']**2) 


In [98]:
sum(counts_subset.iloc[:,5] > 1)

3

In [82]:
properties['Sample_ID'][191]

'HCA_F_BRA8739272'

In [59]:
sum(np.array(counts_subset)[:,i] > 100)

1670

In [40]:
newLOD_2_subset

DSP1001660005641_H10     50.293074
DSP1001660005641_H11     11.596789
DSP1001660005641_H12    135.322994
dtype: float64

In [68]:
[sum(counts[i]) for i in counts.keys()][175]

1557005

In [67]:
np.where(subset_rois)

(array([175, 183, 191]),)

In [84]:
metadata

,ORDER,Sample_ID,panel,construct,instrument_type,read_pattern,expected_neg,ngs_prep,ngs_repnum,pcr_primer_plate,...,human_sample_ID,slide_barcode,age,nuclei,source_ID,Sanger_sampleID,VCDepth,AOI_type,Radial_position,Region
HCA_F_BRA8739289,58.0,DSP1001660005647_A03,WTX,directPCR,NovaSeq,2x50,0.0,Sanger_DSP,1.0,PP08,...,Hob5-BRA-5-FFPE-1-S86,OB00408,14pcw,40.0,13883-46,HCA_F_BRA8739289,0.021415,HOPXpos,1.0,VZ
HCA_F_BRA8739281,57.0,DSP1001660005647_A02,WTX,directPCR,NovaSeq,2x50,0.0,Sanger_DSP,1.0,PP08,...,Hob5-BRA-5-FFPE-1-S86,OB00408,14pcw,272.0,13883-46,HCA_F_BRA8739281,0.021415,EOMESpos,1.0,VZ
HCA_F_BRA8739305,60.0,DSP1001660005647_A05,WTX,directPCR,NovaSeq,2x50,0.0,Sanger_DSP,1.0,PP08,...,Hob5-BRA-5-FFPE-1-S86,OB00408,14pcw,948.0,13883-46,HCA_F_BRA8739305,0.021415,Residual,1.0,VZ
HCA_F_BRA8739297,59.0,DSP1001660005647_A04,WTX,directPCR,NovaSeq,2x50,0.0,Sanger_DSP,1.0,PP08,...,Hob5-BRA-5-FFPE-1-S86,OB00408,14pcw,704.0,13883-46,HCA_F_BRA8739297,0.021415,Ring,1.0,VZ
HCA_F_BRA8739313,61.0,DSP1001660005647_A06,WTX,directPCR,NovaSeq,2x50,0.0,Sanger_DSP,1.0,PP08,...,Hob5-BRA-5-FFPE-1-S86,OB00408,14pcw,220.0,13883-46,HCA_F_BRA8739313,0.004992,Geometric,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HCA_F_BRA8739150,192.0,DSP1001660005640_F09,WTX,directPCR,NovaSeq,2x50,0.0,Sanger_DSP,1.0,PP06,...,Hob7-BRA-5-FFPE-1-S30,OB00199,19pcw,1868.0,14211-15,HCA_F_BRA8739150,NaN,Geometric,NaN,NaN
HCA_F_BRA8739158,193.0,DSP1001660005640_F10,WTX,directPCR,NovaSeq,2x50,0.0,Sanger_DSP,1.0,PP06,...,Hob7-BRA-5-FFPE-1-S30,OB00199,19pcw,1039.0,14211-15,HCA_F_BRA8739158,NaN,Geometric,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PhiX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,PhiX,NaN,NaN,NaN,NaN
